In [1]:
!pip install pdfplumber PyPDF2 reportlab


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\SALMA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached pdfplumber-0.11.6-py3-none-any.whl.metadata (42 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached reportlab-4.3.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached pdfminer_six-20250327-py3-none-any.whl.metadata (4.1 kB)
  Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl.metadata (48 kB)
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
Using cached pdfplumber-0.11.6-py3-none-any.whl (60 kB)
Using cached pdfminer_six-20250327-py3-none-any.whl (5.6 MB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Using cached reportlab-4.3.1-py3-none-any.whl (1.9 MB)
Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl (3.0 MB)
Using cached chardet-5.2.0-py3-none-any.whl (199 kB)
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   - -------------------------------------- 0.2/3.2 MB 2.3 MB/s eta 0:00:02
   ----- ----------------------------------

In [7]:
import re
from datetime import datetime
import pdfplumber
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
import io

# === CONFIGURATION ===
INPUT_PDF = "250215_TSK1_E69 (4).pdf"  # change le nom si besoin
OUTPUT_PDF = "modified.pdf"
THRESHOLD_HOURS = 16

# === EXTRAIRE LES LIGNES DU PDF ===
with pdfplumber.open(INPUT_PDF) as pdf:
    lines = []
    for page in pdf.pages:
        if page.extract_text():
            lines.extend(page.extract_text().splitlines())

# === TROUVER LA PREMIÈRE HEURE DE MISSION ===
first_start = None
for line in lines:
    if "Heure de début de mission" in line:
        match = re.search(r'\b(\d{1,2})[:h](\d{2})\b', line)
        if match:
            first_start = datetime.strptime(f"{match[1]}:{match[2]}", "%H:%M")
            break

if not first_start:
    print("❌ Heure de début non trouvée.")
    exit()

# === TROUVER LA DERNIÈRE HEURE DANS LE DOCUMENT ===
all_times = []
for line in lines:
    if "retour" in line.lower() or "16/02" in line:  # filtre les lignes hors mission
        continue
    found = re.findall(r'\b(\d{1,2})[:h](\d{2})\b', line)
    for h, m in found:
        all_times.append(datetime.strptime(f"{h}:{m}", "%H:%M"))

# Garder uniquement les horaires >= heure de début
valid_times = [t for t in all_times if t >= first_start]
if not valid_times:
    print("❌ Aucune heure de fin trouvée.")
    exit()

last_time = max(valid_times)

# === CALCUL DE LA DURÉE ===
duration = last_time - first_start
total_minutes = duration.total_seconds() / 60
hours = int(total_minutes // 60)
minutes = int(total_minutes % 60)

# === GÉNÉRER LE TEXTE ===
start_str = first_start.strftime("%H:%M")
end_str = last_time.strftime("%H:%M")
duration_str = f"{hours}h{minutes:02d}"

message = (
    f"Heure de début : {start_str}\n"
    f"Heure de fin : {end_str}\n"
    f"Durée totale de la mission : {duration_str}\n"
)

if hours >= THRESHOLD_HOURS:
    message += "La personne a travaillé exceptionnellement longtemps aujourd’hui. Merci pour son implication."
else:
    message += "La personne a accompli sa mission avec implication."

# === CRÉER OVERLAY DU TEXTE À INSÉRER ===
packet = io.BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
can.setFont("Helvetica-Bold", 11)

y = 100
for line in message.split("\n"):
    can.drawString(50, y, line)
    y += 15

can.save()
packet.seek(0)

# === FUSIONNER AVEC LA DERNIÈRE PAGE ===
original = PdfReader(INPUT_PDF)
overlay = PdfReader(packet)
writer = PdfWriter()

for i, page in enumerate(original.pages):
    if i == len(original.pages) - 1:
        page.merge_page(overlay.pages[0])
    writer.add_page(page)

with open(OUTPUT_PDF, "wb") as f:
    writer.write(f)

print(f"✅ PDF modifié avec résumé global généré : {OUTPUT_PDF}")


✅ PDF modifié avec résumé global généré : modified.pdf


In [ ]:
from flask import Flask, request, send_file
from datetime import datetime
import pdfplumber
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
import re
import io

app = Flask(__name__)

@app.route("/traiter", methods=["POST"])
def traiter_pdf():
    file = request.files.get("file")
    if not file:
        return "❌ Aucun fichier reçu.", 400

    lines = []
    with pdfplumber.open(file.stream) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                lines.extend(page.extract_text().splitlines())

    first_start = None
    for line in lines:
        if "Heure de début de mission" in line:
            match = re.search(r'\b(\d{1,2})[:h](\d{2})\b', line)
            if match:
                first_start = datetime.strptime(f"{match[1]}:{match[2]}", "%H:%M")
                break

    if not first_start:
        return "❌ Heure de début non trouvée.", 400

    all_times = []
    for line in lines:
        if "retour" in line.lower() or "16/02" in line:
            continue
        found = re.findall(r'\b(\d{1,2})[:h](\d{2})\b', line)
        for h, m in found:
            all_times.append(datetime.strptime(f"{h}:{m}", "%H:%M"))

    valid_times = [t for t in all_times if t >= first_start]
    if not valid_times:
        last_time = first_start
    else:
        last_time = max(valid_times)

    duration = last_time - first_start
    hours = duration.seconds // 3600
    minutes = (duration.seconds // 60) % 60

    start_str = first_start.strftime("%H:%M")
    end_str = last_time.strftime("%H:%M")
    duration_str = f"{hours}h{minutes:02d}"

    message = (
        f"Heure de début : {start_str}\n"
        f"Heure de fin : {end_str}\n"
        f"Durée totale de la mission : {duration_str}\n"
    )

    if hours >= 16:
        message += "La personne a travaillé exceptionnellement longtemps aujourd’hui 16h. Merci pour son implication."
    else:
        message += "La personne a accompli sa mission avec implication moin de 16h."

    # Création de la page avec le message
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 11)
    y = 100
    for line in message.split("\n"):
        can.drawString(50, y, line)
        y += 15
    can.save()
    packet.seek(0)
    # === Fusionner avec la dernière page ===
    file.stream.seek(0)
    original = PdfReader(file.stream)
    overlay = PdfReader(packet)
    writer = PdfWriter()
    for i, page in enumerate(original.pages):
        if i == len(original.pages) - 1:
            page.merge_page(overlay.pages[0])
        writer.add_page(page)

    output_stream = io.BytesIO()
    writer.write(output_stream)
    output_stream.seek(0)

    return send_file(output_stream, mimetype="application/pdf", as_attachment=True, download_name="modifie.pdf")

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
